# Compare MgSiO3 Models dK09 and S11
* compare resulting ambient properties to experiment-based models of Ghiorso and Lange

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
import numpy as np
import pandas as pd
import pickle

from collections import OrderedDict

import xmeos
from xmeos import models
from xmeos import datamod
from xmeos import modfit
CONSTS = models.CONSTS
import copy

In [ ]:
analysis_file = 'data/analysis.pkl'
with open(analysis_file, 'rb') as f:
    analysis = pickle.load(f)
    
datasets = analysis['datasets']
param_tex_str = analysis['param_tex_str']
eos_electronic = analysis['eos_electronic']



In [ ]:
data_dK09 = datasets['deKoker2009']
data_S11 = datasets['Spera2011']

eos_mod_dK09 = analysis['datamodel_dK09']['eos_mod']
eos_mod_S11 = analysis['datamodel']['eos_mod']
# View data tables
# tbl = data['table']
# tbl

In [ ]:
Texp = 1773


# Properties of MgSiO3 melt
mass_avg = (24.31+28.09+3*16.0)/5.0 # g/(mol atom)
natom=5


rho_conv = 2.35023*14.1735882
ang_cc_mol = CONSTS['ang3percc']/natom/CONSTS['Nmol'] # (ang^3/atom) / (cc/mol)

def exp_properties_Ghiorso(T, Vconv=ang_cc_mol):
    """
    Ghiorso and Kress 2004
    """
    nmol = 2
    dVdT_SiO2 = 1.007e-3
    dVdT_MgO = 2.887e-3
    dVdT = 0.5*dVdT_SiO2 + 0.5*dVdT_MgO
    
    VT0_SiO2 = 26.71
    VT0_MgO = 12.015
    
    VT0 = 0.5*VT0_SiO2 + 0.5*VT0_MgO
    alpha = dVdT/VT0
    
    VT = VT0*np.exp(alpha*(T-1673))
    print(VT)

    c_MgO = 3349.96
    c_SiO2 = 2321.75
    dcdT_MgO = 275.64e-3
    dcdT_SiO2 = 399.34e-3
    
    c = 0.5*(c_MgO + dcdT_MgO*(T-1673)) + 0.5*(c_SiO2 + dcdT_SiO2*(T-1673))
    
    rho = 1e3*mass_avg/(VT*nmol/5)
    betaS = 1e9/rho/c**2
    KS = 1/betaS
    
    
    #betaT_MgO = 0.5*VT_MgO/VT*0.615e-2
    #betaT_SiO2 = 0.5*VT_SiO2/VT*7.15e-2
    #betaT = betaT_MgO + betaT_SiO2
    #K = 1/betaT
    
    props = {}
    props['P'] = 0
    props['T'] = T
    props['V'] = VT*nmol*Vconv
    props['alpha'] = alpha
    props['c'] = c
    props['rho'] = rho
    props['betaS'] = betaS
    props['KS'] = KS
    return props

def exp_properties_Lange(T, Vconv=ang_cc_mol):
    """
    Lange 1997 and Ai and Lange 2004
    """
    nmol = 2

    # V0T = +12.02
    dVdT = (0.5*0+0.5*3.27)*1e-3
    
    VT_SiO2 = 26.86
    VT_MgO = 12.02 + 3.27e-3*(T-1773)
    
    VT = 0.5*VT_SiO2 + 0.5*VT_MgO
    
    alpha = dVdT/VT
    # print(VT)
    
    betaT_MgO = 0.5*VT_MgO/VT*0.615e-2
    betaT_SiO2 = 0.5*VT_SiO2/VT*7.15e-2
    betaT = betaT_MgO + betaT_SiO2
    # print(1/betaT)
    # dVdP = (0.5*-1.922 + 0.5*-.073)
    # K = -VT/dVdP
    K = 1/betaT
 
    
    props = {}
    props['P'] = 0
    props['T'] = T
    props['V'] = nmol*VT*Vconv
    props['dVdT'] = dVdT
    props['alpha'] = alpha
    props['betaT'] = betaT
    props['KT'] = K
    return props
    
    pass
    


props_Ghiorso = exp_properties_Ghiorso(Texp)
props_Lange = exp_properties_Lange(Texp)

display(props_Lange)
display(props_Ghiorso)


In [ ]:
data_1bar = pd.DataFrame(columns=['prop','val','err'])

data_1bar = data_1bar.append({'prop':'V','val':props_Lange['V'],'err':0}, ignore_index=True)
data_1bar = data_1bar.append({'prop':'K_T','val':props_Lange['KT'],'err':0}, ignore_index=True)
data_1bar = data_1bar.append({'prop':'C_V','val':4.2,'err':.2}, ignore_index=True)
data_1bar = data_1bar.append({'prop':'gamma','val':0.37,'err':.14}, ignore_index=True)

data_1bar

In [ ]:
pd.concat?

In [ ]:
props_Lange = analysis['props_Lange']
props_Lange

In [ ]:
eos_mod = eos_mod_dK09
# eos_mod = eos_mod_S11
Tref = 1673
    
    
Vref = eos_mod.volume(0, Tref)
Kref = eos_mod.bulk_mod(Vref, Tref)
alpharef = eos_mod.thermal_exp(Vref, Tref)*1e5
CVref = eos_mod.heat_capacity(Vref, Tref)
gamma_ref = eos_mod.gamma(Vref, Tref)

print('Vref', Vref)
print('Kref', Kref)
print('alpharef', alpharef)
print('CVref', CVref/CONSTS['kboltz'])
print('gammaref', gamma_ref)

In [ ]:
def calc_thermal_data_derivs(data, remove_elec=False, eos_electronic=eos_electronic):
    tbl = data['table']
    dEdT = []
    dPdT = []
    Tavg = []
    Vavg = []

    # plt.figure()
    for Vlbl in np.round(data['V_labels'],2):
        mask = tbl['V_label']==Vlbl
        
        iE = tbl['E'][mask]
        iT = tbl['T'][mask]
        iP = tbl['P'][mask]
        iV = np.round(tbl['V'][mask],decimals=2)
        
        if remove_elec:
            iE -= eos_electronic.energy(iV, iT)
            iP -= eos_electronic.press(iV, iT)
    
        idEdT = np.diff(iE)/np.diff(iT)
        idPdT = np.diff(iP)/np.diff(iT)
    
        dEdT.extend(list(idEdT))
        dPdT.extend(list(idPdT))
    
        Tavg.extend(list(0.5*(np.array(iT[:-1]) + np.array(iT[1:]))))
        Vavg.extend(list(iV[:-1]))
    
        # plt.plot(tbl['T'][mask], tbl['E'][mask], 'ko-')

    Vavg = np.array(Vavg)
    Tavg = np.array(Tavg)
    dEdT = np.array(dEdT)
    dPdT = np.array(dPdT)
    
    thermal_deriv = OrderedDict()
    thermal_deriv['Vavg'] = Vavg
    thermal_deriv['Tavg'] = Tavg
    thermal_deriv['dEdT'] = dEdT
    thermal_deriv['dPdT'] = dPdT
    
    return thermal_deriv

def calc_heat_capacity(data, remove_elec=False, eos_electronic=eos_electronic):
    tbl = data['table']
    Cv = []
    T = []
    V = []

    # plt.figure()
    for Vlbl in np.round(data['V_labels'],2):
        mask = tbl['V_label']==Vlbl
        
        iE = tbl['E'][mask]
        iT = tbl['T'][mask]
        iV = np.round(tbl['V'][mask],decimals=2)
        
        if remove_elec:
            iE -= eos_electronic.energy(iV, iT)
    
        iCv = np.diff(iE)/np.diff(iT)
        # /CONSTS['kboltz']
    
        Cv.extend(list(iCv))
    
        T.extend(list(0.5*(np.array(iT[:-1]) + np.array(iT[1:]))))
        V.extend(list(iV[:-1]))
    
        # plt.plot(tbl['T'][mask], tbl['E'][mask], 'ko-')

    V = np.array(V)
    T = np.array(T)
    Cv = np.array(Cv)
    
    heat_capacity_data = datamod.load_data(
        title='CV_S11', datasource='heat capacity data, Spera 2011',
        V=V, T=T, Cv=Cv, mass_avg=data_dK09['mass_avg'])
    
    # heat_capacity_data = pd.DataFrame()
    # heat_capacity_data['V'] = V
    # heat_capacity_data['T'] = T
    # heat_capacity_data['Cv'] = Cv
    
    
    return heat_capacity_data

def filter_compress_data(data, T0=3000, remove_elec=False, eos_electronic=eos_electronic):
    tbl = data['table'].copy()
    mask_T0 = tbl['T']==T0
    
    if remove_elec:
        # tbl['E'] -= eos_electronic.energy(tbl['V'], tbl['T'])
        tbl['P'] -= eos_electronic.press(tbl['V'], tbl['T'])

        
    tbl_T0 = tbl.loc[mask_T0]
    tbl_T0.reset_index(inplace=True, drop=True)
    
    compress_T0_data = datamod.load_data(
        title='T0_compress_dK09', datasource='3000K isotherm, deKoker2009',
        V=tbl_T0['V'], T=tbl_T0['T'], P=tbl_T0['P'], E=tbl_T0['E'],
        mass_avg=data_dK09['mass_avg'])

    # compress_T0_data = pd.DataFrame()
    # compress_T0_data['V'] = 
    # compress_T0_data['P'] = tbl.loc[mask_T0, 'P']
    # compress_T0_data['E'] = tbl.loc[mask_T0, 'E']
    # compress_T0_data.reset_index(inplace=True, drop=True)

    tbl_therm = tbl.loc[~mask_T0]
    tbl_therm.reset_index(inplace=True, drop=True)
    
    thermal_compress_data = datamod.load_data(
        title='thermal_compress_dK09', datasource='thermal pressure, deKoker2009',
        V=tbl_therm['V'], T=tbl_therm['T'], P=tbl_therm['P'], 
        mass_avg=data_dK09['mass_avg']
    )
    
    # thermal_compress_data = pd.DataFrame()
    # thermal_compress_data['V'] = tbl.loc[~mask_T0, 'V']
    # thermal_compress_data['T'] = tbl.loc[~mask_T0, 'T']
    # thermal_compress_data['P'] = tbl.loc[~mask_T0, 'P']
    # thermal_compress_data['Ptherm'] = 
    # thermal_compress_data.reset_index(inplace=True, drop=True)
    
    return compress_T0_data, thermal_compress_data

def extract_thermal_press(thermal_compress_data, datamodel_T0):
    tbl = thermal_compress_data['table']
    eos_mod = datamodel_T0['eos_mod']
    T0 = np.mean(datamodel_T0['data']['table']['T'])

    dP_therm = tbl['P']-eos_mod.press(tbl['V'], tbl['T'])
    tbl['P_thermal'] = dP_therm
    tbl.drop('P', axis=1, inplace=True)
    pass


In [ ]:
eos_mod_dK09.volume(0,1600)

In [ ]:
print(analysis['props_Lange'])
dPdT_ref = 8.5e-5*19.35*1e3
V_ref = 12.8
T_ref = 1673

In [ ]:
VLange_1773 = 38.88*CONSTS['ang3percc']/CONSTS['Nmol']/5

heat_capacity_1bar = datamod.load_data(
        title='CV_1bar', datasource='heat capacity, vol data, Lange, Stebbins',
        V=[VLange_1773], T=[1773], Cv=[4.2], Cverr=[0.2], mass_avg=data_dK09['mass_avg'])

In [ ]:
heat_capacity_1bar

In [ ]:
compress_T0_data, thermal_compress_data = filter_compress_data(data_dK09, remove_elec=True)
display(compress_T0_data)
display(thermal_compress_data)

In [ ]:
extract_thermal_press(thermal_compress_data, datamodel_T0)

In [ ]:
heat_capacity_data = calc_heat_capacity(data_S11, remove_elec=False)
# heat_capacity_data
tbl = heat_capacity_data['table']

In [ ]:
kind_compress='Vinet'
eos_compress = models.CompressEos(kind=kind_compress, path_const='T')
V0, K0, KP0 = eos_mod_dK09.get_param_values(param_names=['V0','K0','KP0'])

T0 = 3000
eos_compress.set_param_values([V0,K0,KP0],param_names=['V0','K0','KP0'])

In [ ]:
Vmod = np.linspace(6,16,101)

print(V0,K0,KP0)
Pmod0 = eos_compress.press(Vmod, T0)

In [ ]:
datamodel_T0 = datamod.init_datamodel(compress_T0_data, eos_compress)


fit_calcs = ['compress']
# fix_params = ['S0','Cvlimfac','mexp']
# datamodel['eos_mod'].set_param_values([3/5,1], param_names=['mexp','Cvlimfac'])
eos_fit_params = datamod.select_eos_fit_params(datamodel_T0, fit_calcs)
print(eos_fit_params)

In [ ]:
compress_prior = modfit.ModelPDF(['V0', 'K0', 'KP0'], 
                                 [15,  12,  7.5], [4, 5, 4])
datamod.set_model_pdf(datamodel_T0, compress_prior)

In [ ]:
# datamod.impose_prior_constraints(datamodel_T0, ['V0','K0','KP0'], 
#                                  [15,  12,  7.5], [4, 5, 4])

In [ ]:
Pbounds_prior, Pdraw_prior = datamod.model_pdf_prediction(Vmod, T0, 'press', datamodel_T0, Ndraw=1000)

In [ ]:
datamod.fit(datamodel_T0, apply_bulk_mod_wt=True, ignore_datatypes=['E'])
Pmod_fit = datamodel_T0['eos_mod'].press(Vmod, T0)

In [ ]:
display(datamodel_T0['model_pdf'].param_table)
display(datamodel_T0['model_pdf'].param_corr)

In [ ]:
datamodel_T0['eos_mod'].param_names

In [ ]:
Pbounds, Pdraw = datamod.model_pdf_prediction(Vmod, T0, 'press', datamodel_T0, Ndraw=1000)
# Pbounds, Pdraw = datamod.model_pdf_prediction(Vmod, T0, 'press', datamodel_T0, 
#                                               Ndraw=1000, percentile=[2.5,97.5])

In [ ]:

plt.figure()
plt.fill_between(Vmod, Pbounds_prior[0], Pbounds_prior[-1], color=[1,.9,.9])
plt.fill_between(Vmod, Pbounds[0], Pbounds[-1], color=[.7,.7,.7])
# plt.plot(Vmod, Pmod_fit, 'r-')
plt.plot(compress_T0_data['table']['V'], compress_T0_data['table']['P'], 'ko')
plt.ylim([-5,200])


In [ ]:
RT_eos = copy.deepcopy(analysis['datamodel']['eos_mod'])
RT_eos.apply_electronic = False
RT_eos.heat_capacity(10,3500)/CONSTS['kboltz']
Cv_mod = RT_eos.heat_capacity(tbl['V'], tbl['T'])
Cv_ref_mod = RT_eos.heat_capacity(V_ref, 1773)/CONSTS['kboltz']

In [ ]:
datamodel_Cv = datamod.init_datamodel(heat_capacity_data, RT_eos)

fit_calcs = ['bcoef', 'thermal']
fix_params = ['Cvlimfac','mexp', 'V0']
# datamodel['eos_mod'].set_param_values([3/5,1], param_names=['mexp','Cvlimfac'])
eos_fit_params = datamod.select_eos_fit_params(datamodel_Cv, fit_calcs, fix_params=fix_params)
eos_fit_params

In [ ]:
param_init = [1, 1, 1, 1, 1]
# param_init = [1.009, 0.7, -1.4, -15.2, -17.3]

Cv_prior = modfit.ModelPDF(
    ['_bcoef_0', '_bcoef_1', '_bcoef_2', '_bcoef_3', '_bcoef_4'], 
    param_init, [0.3,10,10,10,10])


datamod.set_model_pdf(datamodel_Cv, Cv_prior)

In [ ]:
model_pdf_all = datamodel_T0['model_pdf'].expand(
    datamodel_Cv['model_pdf'])
datamod.set_model_pdf(datamodel_Cv, model_pdf_all)

In [ ]:
print(model_pdf_all.param_names)
print(np.round(model_pdf_all.param_corr, decimals=2))
print(np.round(model_pdf_all.param_errors, decimals=2))

plt.figure()
plt.imshow(model_pdf_all.param_corr, cmap='seismic')
plt.clim(-1,1)

In [ ]:
datamod.fit(datamodel_Cv, apply_bulk_mod_wt=False)
#Cvmod_fit = datamodel_Cv['eos_mod'].heat_capacity(Vmod, T0)
eos_fit = datamodel_Cv['eos_mod']
Cv_fit = eos_fit.heat_capacity(tbl['V'], tbl['T'])
Cv_ref_fit = eos_fit.heat_capacity(V_ref, 1773)/CONSTS['kboltz']

In [ ]:
datamodel_Cv['model_pdf'].param_table

In [ ]:
plt.figure()
plt.imshow(datamodel_Cv['model_pdf'].param_corr, cmap='seismic')
plt.clim(-1,1)
plt.colorbar()

In [ ]:
Vuniq = np.unique(tbl['V'])
Tmod = np.linspace(1500,5000,101)
Vbnds = [5,15]

def display_Cv_data_fit(Vmodel, Tmodel, Vbounds, tbl,
                        Vthresh=10, dV_sign=+1):
    
    
    if dV_sign>0:
        Vmask = tbl['V']>=Vthresh
        Vmodel_show = Vmodel[Vmodel>=Vthresh]
        if V_ref >= Vthresh:
            show_ref = True
        else:
            show_ref = False
    elif dV_sign<0:
        Vmask = tbl['V']<Vthresh
        Vmodel_show = Vmodel[Vmodel<Vthresh]
        if V_ref < Vthresh:
            show_ref = True
        else:
            show_ref = False
    else:
        Vmask = np.tile(True, len(tbl['V']))
        Vmodel_show = Vmodel
        show_ref = True
        
        
    cmap = plt.get_cmap('gist_earth')
    #cmap = plt.get_cmap('magma')
    # cmap = plt.get_cmap('viridis')
    #cmap = plt.get_cmap('magma')
    col_ref = (V_ref-Vbounds[0])/(Vbounds[1]-Vbounds[0])


    plt.figure()
    plt.scatter(tbl['T'][Vmask],tbl['Cv'][Vmask]/CONSTS['kboltz'], 
                c=tbl['V'][Vmask], cmap=cmap, edgecolor='k', s=100)

    plt.clim(Vbnds)
    
    for iV in Vmodel_show:
        icol = (iV-Vbnds[0])/(Vbnds[-1]-Vbnds[0])
        plt.plot(Tmod, eos_fit.heat_capacity(iV, Tmod)/CONSTS['kboltz'],
             '-', color=cmap(icol))
    
    
    if show_ref:
        plt.plot(1773,4.2,'*', color=cmap(col_ref))


    plt.ylabel('Cv/Nk')
    plt.xlabel('Temperature [K]')
    plt.colorbar(label='V')
    return


In [ ]:
Vmodel = np.unique(tbl['V'])
Tmodel = np.linspace(1500,5000,101)
Vbounds = [5,15]

display_Cv_data_fit(Vmodel, Tmodel, Vbounds, tbl, Vthresh=10, dV_sign=+1)
display_Cv_data_fit(Vmodel, Tmodel, Vbounds, tbl, Vthresh=10, dV_sign=-1)

In [ ]:
display_Cv_data_fit(Vmodel, Tmodel, Vbounds, tbl, dV_sign=0)

In [ ]:
dP_therm_Lange = (props_Lange['T']-T0)*props_Lange['KT']*props_Lange['alpha']


In [ ]:
RTpress_eos = copy.deepcopy(analysis['datamodel']['eos_mod'])
RTpress_eos.apply_electronic = False

In [ ]:
datamodel_P_thermal = datamod.init_datamodel(thermal_compress_data, RTpress_eos)

fit_calcs = [ 'gamma']
fix_params = ['V0']
# datamodel['eos_mod'].set_param_values([3/5,1], param_names=['mexp','Cvlimfac'])
eos_fit_params = datamod.select_eos_fit_params(datamodel_P_thermal, fit_calcs, fix_params=fix_params)
eos_fit_params

In [ ]:
RTpress_eos

In [ ]:
param_init = [1, -1]
# param_init = [1.009, 0.7, -1.4, -15.2, -17.3]

gamma_prior = modfit.ModelPDF(
    ['gamma0', 'gammap0'], param_init, [2,2])


datamod.set_model_pdf(datamodel_P_thermal, gamma_prior)

In [ ]:
model_pdf_all = datamodel_Cv['model_pdf'].expand(
    datamodel_P_thermal['model_pdf'])
datamod.set_model_pdf(datamodel_P_thermal, model_pdf_all)

In [ ]:
print(model_pdf_all.param_names)
print(np.round(model_pdf_all.param_corr, decimals=2))
print(np.round(model_pdf_all.param_errors, decimals=2))

plt.figure()
plt.imshow(model_pdf_all.param_corr, cmap='seismic')
plt.clim(-1,1)

In [ ]:
datamod.fit(datamodel_P_thermal)
eos_fit = datamodel_P_thermal['eos_mod']

T0 = eos_fit.refstate.ref_temp()

P_thermal_fit = eos_fit.press(tbl['V'], tbl['T'])-eos_fit.press(tbl['V'], T0)


In [ ]:
plt.figure()
tbl = thermal_compress_data['table']


plt.scatter(tbl['V'], tbl['P_thermal'], c=tbl['T']-T0, cmap=plt.get_cmap('coolwarm',11))
plt.clim(-5500,5500)
plt.scatter(tbl['V'], P_thermal_fit, c=tbl['T']-T0, marker='^', edgecolor='k',cmap=plt.get_cmap('coolwarm',11))
plt.clim(-5500,5500)
plt.plot(props_Lange['V'], dP_therm_Lange, 'kx')
plt.colorbar()

plt.figure()

plt.scatter(tbl['T']-T0, tbl['P_thermal'], c=tbl['V'],  cmap=plt.get_cmap('jet'))
plt.clim(5,15)
plt.scatter(tbl['T']-T0, P_thermal_fit, c=tbl['V'],  marker='^', edgecolor='k', cmap=plt.get_cmap('jet'))
plt.clim(5,15)
plt.plot( props_Lange['T']-T0, dP_therm_Lange,  'kx')
plt.plot(0,0,'k+')
plt.colorbar()

In [ ]:
datamodel_P_thermal['model_pdf'].param_table

In [ ]:
plt.figure()
plt.imshow(datamodel_P_thermal['model_pdf'].param_corr, cmap='seismic')
plt.clim(-1,1)
plt.colorbar()

In [ ]:
def load_shock_data(datafile='data/shock-data-Mosenfelder2009.csv'):
    shock_dat = pd.read_csv(datafile,delimiter=',')
    melt_en_mask = ((shock_dat['Starting Material']=='Enstatite')&(shock_dat['Phase State']=='melt'))
    melt_glass_mask = ((shock_dat['Starting Material']=='Glass')&(shock_dat['Phase State']=='melt'))

    melt_enpor_mask = ((shock_dat['Starting Material']=='Porous Enstatite')&(shock_dat['Phase State']=='melt'))
    melt_oxmix_mask = ((shock_dat['Starting Material']=='Oxide mix')&(shock_dat['Phase State']=='melt'))

    shock_dat_melt_glass = shock_dat.loc[melt_glass_mask]
    shock_dat_melt_en = shock_dat.loc[melt_en_mask]
    
    return shock_dat_melt_glass, shock_dat_melt_en

def get_melt_rxn_info():    
    shock_dat_melt_glass, shock_dat_melt_en = load_shock_data()
    
    # Data on pre-melting behavior of enstatite from 
    
    Tmelt = 1816
    

    # Enstatite Hugoniot
    rhoinit_en=np.mean(shock_dat_melt_en['rho0'])
    # rhofaclims_en = [1.59, 1.84]
    rhofaclims_en = np.array([1.6, 1.8])
    # Etrans_en = 2.192455e6*1e-3/const_d['JperHa']*const_d['eVperHa']*eos_d['param_d']['mass_avg']/const_d['Nmol']
    
    #Tmelt_en = 300
    Efus_en = 73.2*1e3/5/CONSTS['JperHa']*CONSTS['eVperHa']/CONSTS['Nmol']
    
    # This should probably be integrated from 300K not 273....
    # Eheat_en = 126*(Tmelt-273)/5/CONSTS['JperHa']*CONSTS['eVperHa']/CONSTS['Nmol']
    Eheat_en = (126*(Tmelt-273) - 80*(300-273))/5/CONSTS['JperHa']*CONSTS['eVperHa']/CONSTS['Nmol']
    Etrans_en = Eheat_en+Efus_en

    melt_rxn_en = {}
    melt_rxn_en['Tmelt'] = Tmelt    
    melt_rxn_en['rhoinit'] = rhoinit_en
    melt_rxn_en['rhofaclims'] = rhofaclims_en
    melt_rxn_en['Etrans'] = Etrans_en

    # Glass Hugoniot
    rhoinit_glass = np.mean(shock_dat_melt_glass['rho0'])
    # rhofaclims_glass = [1.65, 1.94]
    rhofaclims_glass = [1.65, 1.9]
    # Etrans_glass = 1.862455e6*1e-3/const_d['JperHa']*const_d['eVperHa']*eos_d['param_d']['mass_avg']/const_d['Nmol']
    
    # delE_glass = (2.192455e6-1.862455e6)*1e-3/CONSTS['JperHa']*CONSTS['eVperHa']*eos_mod.molar_mass/CONSTS['Nmol']
    # Etrans_glass = Etrans_en - delE_glass
    
    Efus_glass = Efus_en = (73.2-42.1)*1e3/5/CONSTS['JperHa']*CONSTS['eVperHa']/CONSTS['Nmol']
    Etrans_glass = Eheat_en + Efus_glass
    
    melt_rxn_glass = {}
    melt_rxn_glass['Tmelt'] = Tmelt    
    melt_rxn_glass['rhoinit'] = rhoinit_glass
    melt_rxn_glass['rhofaclims'] = rhofaclims_glass
    melt_rxn_glass['Etrans'] = Etrans_glass
    
    return melt_rxn_glass, melt_rxn_en

def calc_hugoniot(eos_mod, melt_rxn, Tinit=300):
    hugoniot = eos_mod.hugoniot(
        melt_rxn['rhofaclims'], melt_rxn['rhoinit'], Tinit, 
        Etrans=melt_rxn['Etrans'], Ttrans=melt_rxn['Tmelt'],
        isobar_trans=True)
    
    return hugoniot

In [ ]:
shock_dat_melt_glass, shock_dat_melt_en = load_shock_data()
melt_rxn_glass, melt_rxn_en = get_melt_rxn_info()

hugoniot_glass = calc_hugoniot(eos_fit, melt_rxn_glass)
hugoniot_en = calc_hugoniot(eos_fit, melt_rxn_en)



In [ ]:
def plot_hugoniot(shock_dat_melt_glass, shock_dat_melt_en,
                  hugoniot_glass_S11, hugoniot_en_S11,
                  hugoniot_glass_dK09, hugoniot_en_dK09,
                  Plim=[50,225], col_glass=[0,0,0], col_en=[.5,.5,.5]):
    
    f, ax_a = plt.subplots(2, 1, sharex='col')

    ax_a[0].errorbar(shock_dat_melt_glass['P'], shock_dat_melt_glass['rho'],
                     xerr=shock_dat_melt_glass['P err'],
                     yerr=shock_dat_melt_glass['rho err'],
                     fmt='.',color=col_glass)
    ax_a[0].errorbar(shock_dat_melt_en['P'], shock_dat_melt_en['rho'],
                     xerr=shock_dat_melt_en['P err'],
                     yerr=shock_dat_melt_en['rho err'],
                     fmt='.',color=col_en)

    ax_a[0].plot(hugoniot_glass_S11['P_a'],hugoniot_glass_S11['rho_a'],
                 '-',color=col_glass)
    ax_a[0].plot(hugoniot_en_S11['P_a'],hugoniot_en_S11['rho_a'],
                 '-',color=col_en)

    ax_a[0].plot(hugoniot_glass_dK09['P_a'],hugoniot_glass_dK09['rho_a'],
                 '--',color=col_glass)
    ax_a[0].plot(hugoniot_en_dK09['P_a'],hugoniot_en_dK09['rho_a'],
                 '--',color=col_en)

    ax_a[0].set_xlim(Plim[0],Plim[1])
    ax_a[0].set_ylabel(u'Density [g/cm$^3$]')

    
    
    ax_a[1].plot(hugoniot_glass_S11['P_a'],hugoniot_glass_S11['T_a'],
                 '-',color=col_glass)
    ax_a[1].plot(hugoniot_en_S11['P_a'],hugoniot_en_S11['T_a'],
                 '-',color=col_en)

    ax_a[1].plot(hugoniot_glass_dK09['P_a'],hugoniot_glass_dK09['T_a'],
                 '--',color=col_glass)
    ax_a[1].plot(hugoniot_en_dK09['P_a'],hugoniot_en_dK09['T_a'],
                 '--',color=col_en)

    ax_a[1].errorbar(shock_dat_melt_glass['P'], shock_dat_melt_glass['TH'],
                     xerr=shock_dat_melt_glass['P err'],
                     yerr=shock_dat_melt_glass['TH err'],
                     fmt='.',color=col_glass)
    ax_a[1].errorbar(shock_dat_melt_en['P'], shock_dat_melt_en['TH'],
                     xerr=shock_dat_melt_en['P err'],
                     yerr=shock_dat_melt_en['TH err'],
                     fmt='.',color=col_en)
    
    ax_a[1].set_xlim(Plim[0],Plim[1])
    ax_a[1].set_xlabel('Pressure  [GPa]')
    ax_a[1].set_ylabel('Temperature  [K]')
    ax_a[1].set_ylim(2000,8000)

    ax_a[0].text(80,4.78,'glass hugoniot',fontsize=12,color=col_glass,
                 verticalalignment='center',horizontalalignment='center',rotation=20)
    ax_a[0].text(150,5.5,'enstatite hugoniot',fontsize=12,color=col_en,
                 verticalalignment='center',horizontalalignment='center',rotation=15)

    ax_a[0].axvspan(136,Plim[1],color = [.95,.95,.95])
    ax_a[1].axvspan(136,Plim[1],color = [.95,.95,.95])


    ax_a[1].text(90,7000,'Terrestrial Mantle\nRegion',fontsize=14,color='k',
                 verticalalignment='top',horizontalalignment='center',weight='bold')

    plt.draw()
    plt.tight_layout(h_pad=.15)
    pass

In [ ]:
plot_hugoniot(shock_dat_melt_glass, shock_dat_melt_en,
                  hugoniot_glass, hugoniot_en,
                  hugoniot_glass, hugoniot_en)

In [ ]:
Vbnds = [5,15]

cmap = plt.get_cmap('gist_earth')
cmap = plt.get_cmap('magma')
# cmap = plt.get_cmap('viridis')
col_ref = (V_ref-Vbnds[0])/(Vbnds[1]-Vbnds[0])

plt.figure()
plt.scatter(thermal_deriv_S11['Tavg'],thermal_deriv_S11['dEdT']/CONSTS['kboltz'], 
            c=thermal_deriv_S11['Vavg'], cmap=cmap, edgecolor='k', s=100)
plt.scatter(thermal_deriv_dK09['Tavg'],thermal_deriv_dK09['dEdT']/CONSTS['kboltz'],
            c=thermal_deriv_dK09['Vavg'], marker='s', cmap=cmap, edgecolor='k', s=100)
plt.plot(1773,4.2,'*', color=cmap(col_ref))
plt.ylabel('Cv/Nk')
plt.colorbar(label='V')

plt.figure()
plt.scatter(thermal_deriv_S11['Tavg'],thermal_deriv_S11['dPdT']*1e3, 
            c=thermal_deriv_S11['Vavg'], cmap=cmap, edgecolor='k', s=100,
            vmin=Vbnds[0],vmax=Vbnds[1])

plt.scatter(thermal_deriv_dK09['Tavg'],thermal_deriv_dK09['dPdT']*1e3, 
            c=thermal_deriv_dK09['Vavg'], marker='s', cmap=cmap, edgecolor='k', s=100,
           vmin=Vbnds[0],vmax=Vbnds[1])


plt.plot(T_ref, dPdT_ref,'*', color=cmap(col_ref))
plt.ylabel('dP/dT  [GPa/1000K]')
plt.colorbar(label='V')

In [ ]:
plt.figure()
plt.scatter(np.random.randn(len(thermal_deriv_S11['dPdT'])),thermal_deriv_S11['dPdT']*1e3, 
            c=thermal_deriv_S11['Vavg'], cmap=cmap, edgecolor='k', s=100,
            vmin=Vbnds[0],vmax=Vbnds[1])

plt.scatter(np.random.randn(len(thermal_deriv_dK09['dPdT'])),thermal_deriv_dK09['dPdT']*1e3, 
            c=thermal_deriv_dK09['Vavg'], marker='s', cmap=cmap, edgecolor='k', s=100,
           vmin=Vbnds[0],vmax=Vbnds[1])


plt.plot(dPdT_ref,'*', color=cmap(col_ref))
plt.ylabel('dP/dT  [GPa/1000K]')
plt.colorbar(label='V')

In [ ]:

plt.figure()
Vavg = np.unique(thermal_deriv_S11['Vavg'])
for iV in Vavg:
    icol = (iV-Vbnds[0])/(Vbnds[1]-Vbnds[0])
    #icol = (iV-Vavg[0])/(Vavg[-1]-Vavg[0])
    mask = thermal_deriv_S11['Vavg']==iV
    plt.plot(thermal_deriv_S11['Tavg'][mask],thermal_deriv_S11['dEdT'][mask]/CONSTS['kboltz'], 
            'o-', markeredgecolor='k',color = cmap(icol))

Vavg = np.unique(thermal_deriv_dK09['Vavg'])
for iV in Vavg:
    icol = (iV-Vbnds[0])/(Vbnds[1]-Vbnds[0])
    #icol = (iV-Vavg[0])/(Vavg[-1]-Vavg[0])
    mask = thermal_deriv_dK09['Vavg']==iV
    plt.plot(thermal_deriv_dK09['Tavg'][mask],thermal_deriv_dK09['dEdT'][mask]/CONSTS['kboltz'], 
            's--', markeredgecolor='k',color = cmap(icol))
    
plt.ylabel('Cv/Nk')
plt.xlabel('T  [K]')
#plt.colorbar(label='V')

In [ ]:
plt.figure()
Vavg = np.unique(thermal_deriv_S11['Vavg'])
for iV in Vavg:
    # icol = (iV-Vavg[0])/(Vavg[-1]-Vavg[0])
    icol = (iV-Vbnds[0])/(Vbnds[1]-Vbnds[0])
    mask = thermal_deriv_S11['Vavg']==iV
    plt.plot(thermal_deriv_S11['Tavg'][mask],thermal_deriv_S11['dPdT'][mask]*1e3, 
            'o-', markeredgecolor='k',color = cmap(icol))

Vavg = np.unique(thermal_deriv_dK09['Vavg'])
for iV in Vavg:
    icol = (iV-Vbnds[0])/(Vbnds[1]-Vbnds[0])
    #icol = (iV-Vavg[0])/(Vavg[-1]-Vavg[0])
    mask = thermal_deriv_dK09['Vavg']==iV
    plt.plot(thermal_deriv_dK09['Tavg'][mask],thermal_deriv_dK09['dPdT'][mask]*1e3, 
            's--', markeredgecolor='k',color = cmap(icol))
    
plt.ylabel('dP/dT  [GPa/1000K]')
plt.xlabel('T [K]')
#plt.colorbar(label='V')

In [ ]:
tbl_dK09 = data_dK09['table']
mask_T3K = tbl_dK09['T']==3000

compress_T0_data = pd.DataFrame()
compress_T0_data['V'] = tbl_dK09.loc[mask_T3K, 'V']
compress_T0_data['P'] = tbl_dK09.loc[mask_T3K, 'P']

compress_T0_data.reset_index(inplace=True, drop=True)

thermal_compress_data = tbl_dK09.loc[~mask_T3K]
thermal_compress_data.reset_index(inplace=True, drop=True)
thermal_compress_data

